In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import required libraries
import pandas as pd
import numpy as np
import sklearn
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score

import tensorflow as tf

import tensorflow.keras.backend as K
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense , LSTM, Dropout
from tensorflow.keras.models import Sequential, load_model
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
import math
# import xgboost
import time
from tqdm import tqdm

# Setting seed for reproducibility
import random
np.random.seed(1234)
PYTHONHASHSEED = 1234
tf.random.set_seed(1234)

In [ ]:
columns = ['unit_number','time_in_cycles','setting_1','setting_2',
           'TRA','T2','T24','T30','T50','P2','P15','P30','Nf',
           'Nc','epr','Ps30','phi','NRf','NRc','BPR','farB',
           'htBleed','Nf_dmd','PCNfR_dmd','W31','W32' 
          ]
# '../input/phmdataset'
train_orig = pd.read_csv('../input/phmdataset/train_orig.csv', header=None)
test_x_orig = pd.read_csv('../input/phmdataset/test_x_orig.csv', header=None)
test_y_orig = pd.read_csv('../input/phmdataset/test_y_orig.csv', header=None)

train_orig.columns = columns
test_x_orig.columns = columns

In [ ]:
from sklearn import pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Combine the X values to normalize them, 
all_data_orig = pd.concat([train_orig, test_x_orig])
# all_data = all_data[feature_cols]
# all_data[feature_cols] = normalize(all_data[feature_cols].values)

mmscaler = MinMaxScaler(feature_range=(-1, 1))
all_data = all_data_orig.copy()
scaler=pipeline.Pipeline(steps=[
     ('minmax', MinMaxScaler(feature_range=(-1, 1))),
     ('remove_constant', VarianceThreshold())
])

feature_cols = columns[2:]
all_data = all_data_orig.copy()
all_data = np.concatenate(
    [
        all_data.iloc[:, :2], 
        scaler.fit_transform(all_data.iloc[:,2:])
    ], 
    axis=1)


# then split them back out
train = all_data[0:train_orig.shape[0], :]
test = all_data[train_orig.shape[0]:, :]

# Make engine numbers and days zero-indexed, for everybody's sanity
train[:, 0:2] -= 1
test[:, 0:2] -= 1

In [ ]:
from tensorflow.keras.utils import to_categorical
def get_labels(x, max_val, bins=0.05):
#     print(x, max_val)
#     max_val = max(x)
    
    n = 1 / bins + 1
    interval = max_val * bins
    if 0 <= x <= interval:
        return 0 if x < interval * 0.5 else 1
    for i in range(1, 21):
        if interval * i < x <= interval * (i + 1):
            return i if x < interval * i + 0.5 * interval else i + 1
    return 20
    

def prepare_data(engine, time, x, window, mask_value, is_test=False):
    n_engines = 100
    out_y_clf = []
    out_y_reg = []
    out_x = []
    d = x.shape[1]
    
    for i in range(n_engines):
        max_run_time = int(np.max(time[engine==i])) + 1
        if not is_test:
            start = 0
        else:
            start = max_run_time - 1
        
        this_x = []
        this_y = []
        
        for j in range(start, max_run_time):
            engine_x = x[engine==i]
            if not is_test:
                # 不用再加上一个1了
                labels = get_labels(max_run_time - j, max_run_time)
                this_y.append(np.array([labels]))
                out_y_reg.append(np.array((max_run_time - j,), ndmin=2))
            # x_temp = np.zeros((1, window, d))
            x_temp = np.zeros((window, d))
            x_temp += mask_value
            
            # x_temp[:, max(0, window - j - 1):window, :] = engine_x[max(0, j - window + 1):j + 1, :]
            x_temp[max(0, window - j - 1):window, :] = engine_x[max(0, j - window + 1):j + 1, :]
            this_x.append(x_temp)
        if not is_test:
            out_y_clf.append(this_y)
        out_x.append(this_x)
        
    out_x = np.concatenate(out_x)
    if not is_test:
        out_y_clf = np.concatenate(out_y_clf)
        out_y_clf = to_categorical(out_y_clf, num_classes=21, dtype='int16')
        out_y_reg = np.concatenate(out_y_reg)
    return out_x, (out_y_clf, out_y_reg)

mask_value = -99
train_x, train_y = prepare_data(
    engine=train[:, 0], time=train[:, 1],
    x=train[:, 2:],
    window=50,
    is_test=False,
    mask_value=mask_value
)

test_x, _ = prepare_data(
    engine=test[:, 0], time=test[:, 1],
    x=test[:, 2:],
    window=50,
    is_test=True,
    mask_value=mask_value  
)

In [ ]:
train_y_clf, train_y_reg = train_y
train_x.shape, train_y_clf.shape, train_y_reg.shape

In [ ]:
from tensorflow.keras.layers import Dense, LSTM, Masking
from tensorflow.keras.layers import Layer, concatenate
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.losses import CategoricalCrossentropy

from tensorflow.keras.metrics import MeanAbsoluteError, Accuracy

inputs = tf.keras.Input(shape=(train_x.shape[1], train_x.shape[2]))
x = Masking(mask_value=50)(inputs)
x = LSTM(50)(x)

out1 = Dense(50, activation='relu')(x)
out1 = Dense(21, activation='softmax', name='output_1')(out1)

out2 = Dense(50, activation='relu')(x)
out2 = Dense(1, activation='relu', name='output_2')(out2)

model = tf.keras.Model(inputs=[inputs], outputs=[out1, out2])

model.compile(
    optimizer=tf.keras.optimizers.RMSprop(0.003),
    loss={
        'output_1': CategoricalCrossentropy(),
        'output_2': MeanSquaredError()
    },
    loss_weights={
        'output_1': 0.95,
        'output_2': 0.05
    },
    metrics={
        'output_1': Accuracy(),
        'output_2': MeanAbsoluteError()
    }
)


checkpoint_filepath = 'MTL-1130-0.001-{epoch:02d}-{val_loss:.2f}.hdf5'
h_6_m5 = model.fit(
    train_x, train_y, 
    epochs=400, batch_size=64, 
    validation_split=0.02,
#     validation_data = (test_x, test_y),
    callbacks = [
#         keras.callbacks.EarlyStopping(
#             monitor='loss', min_delta=0, patience=20, verbose=0, mode='min'),
#         tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath ,monitor="val_accuracy", verbose=0,save_best_only=False),
        tf.keras.callbacks.TerminateOnNaN(),
#         tbCallBack
    ]
)

In [ ]:
# h_6_m5
# h_6_m5
keys = ['loss', 'val_loss', 
        'output_1_loss', 'val_output_1_loss', 
        'output_2_loss', 'val_output_2_loss', 
        'output_1_accuracy',  'val_output_1_accuracy', 
        'output_2_mean_absolute_error', 'val_output_2_mean_absolute_error']
fig, axs = plt.subplots(5, 2, figsize=(8, 12))
fig.suptitle('2021.11.30.RMSprop(0.005),loss_weight:0.95, 0.05, epochs=400,batch_size=64')
for idx in range(10):
    i, j = idx // 2, idx % 2
    axs[i][j].plot(h_6_m5.history[keys[idx]][:250])
    axs[i][j].set_title(keys[idx])
axs[3][0].hlines(0.5, xmin=0, xmax=len(h_6_m5.history[keys[idx]]), color='red')

In [ ]:
y_pred = model.predict(train_x)

In [ ]:
e = np.abs(y_pred[1] - train_y[1])

In [ ]:
error = y_pred[1] - train_y[1]
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
axs[0].plot(error)
axs[0].set_title('error, error=y_pred_clf - y_true_clf')

axs[1].plot(error[:18000])
axs[1].set_title('error[:18000], error=y_pred_clf - y_true_clf')

In [ ]:
len(error[error>10]) / len(error)

In [ ]:
y_true_clf, y_pred_clf = np.argmax(train_y[0], axis=1), np.argmax(y_pred[0], axis=1)

In [ ]:
type(y_true_clf)

In [ ]:
pd.DataFrame([y_true_clf, y_pred_clf])

In [ ]:
# plt.hist(e[:18000], 100, density=True, facecolor='g', alpha=0.75)
n, bins, patches = plt.hist(error[:15000], 100, density=True, facecolor='g', alpha=0.75)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
n, bins, patches = axs[0].hist(error, 100, density=True, facecolor='g', alpha=0.75)
axs[0].set_title('error')

n, bins, patches = axs[1].hist(error[:18000], 100, density=True, facecolor='r', alpha=0.75)
axs[1].set_title('error[:18000]')

fig.suptitle('error = y_pred_clf - y_true_clf')

In [ ]:
test_y_orig.shape

In [ ]:
test_y_orig[0]

In [ ]:
def get_labels(x, max_val, bins=0.05):
#     print(x, max_val)
#     max_val = max(x)
#     print(type(x))
    
    n = 1 / bins + 1
    interval = max_val * bins
#     print(x.values)
    x = int(x.values)
    if 0 <= x <= interval:
        return 0 if x < interval * 0.5 else 1
    for i in range(1, 21):
        if interval * i < x <= interval * (i + 1):
            return i if x < interval * i + 0.5 * interval else i + 1
    return 20

In [ ]:
a = pd.Series([1, 2, 3])
def func(x, const):
    return x + const
# a.apply(get_labels, args=(145,))

In [ ]:
test_y_reg.shape

In [ ]:
test_y_reg = test_y_orig.copy()
max_val = int(test_y_reg.max())

test_y_clf = test_y_reg.apply(get_labels, args=(max_val,), axis=1)
test_y_clf = to_categorical(test_y_clf, num_classes=21, dtype='int16')
test_y_clf = np.argmax(test_y_clf, axis=1)

In [ ]:
test_y_pred = model.predict(test_x)

In [ ]:
test_y_pred_clf, test_y_pred_reg = test_y_pred
test_y_pred_clf = np.argmax(test_y_pred_clf, axis=1)

In [ ]:
test_y_clf.shape, test_y_pred_clf.shape

In [ ]:
pd.DataFrame([test_y_clf, test_y_pred_clf])

In [ ]:
error_test = test_y_clf - test_y_pred_clf
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
axs[0].plot(error_test)
axs[0].set_title('the curve of error_test')
n, bins, patches = axs[1].hist(error_test, 20)
axs[1].set_title('the distribution or error_test')


In [ ]:
len(error_test[error_test > 4]) / len(error_test)

In [ ]:
# h_6_m5
keys = ['loss', 'val_loss', 
        'output_1_loss', 'val_output_1_loss', 
        'output_2_loss', 'val_output_2_loss', 
        'output_1_accuracy',  'val_output_1_accuracy', 
        'output_2_mean_absolute_error', 'val_output_2_mean_absolute_error']
fig, axs = plt.subplots(5, 2, figsize=(8, 12))
fig.suptitle('2021.11.30.RMSprop(0.005),loss_weight:0.95, 0.05, epochs=400,batch_size=64')
for idx in range(10):
    i, j = idx // 2, idx % 2
    axs[i][j].plot(h_6_m5.history[keys[idx]])
    axs[i][j].set_title(keys[idx])

In [ ]:
# h_6_m5
keys = ['loss', 'val_loss', 
        'output_1_loss', 'val_output_1_loss', 
        'output_2_loss', 'val_output_2_loss', 
        'output_1_accuracy',  'val_output_1_accuracy', 
        'output_2_mean_absolute_error', 'val_output_2_mean_absolute_error']
fig, axs = plt.subplots(5, 2, figsize=(8, 12))
fig.suptitle('2021.11.30.RMSprop(0.005),loss_weight:0.95, 0.05, epochs=400,batch_size=64')
for idx in range(10):
    i, j = idx // 2, idx % 2
    axs[i][j].plot(h_6_m5.history[keys[idx]][50:])
    axs[i][j].set_title(keys[idx])

In [ ]:
h_6_m2 = model.fit(
    train_x, train_y, 
    epochs=100, batch_size=64, 
    validation_split=0.02,
#     validation_data = (test_x, test_y),
    callbacks = [
#         keras.callbacks.EarlyStopping(
#             monitor='loss', min_delta=0, patience=20, verbose=0, mode='min'),
#         tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath ,monitor="val_accuracy", verbose=0,save_best_only=False),
        tf.keras.callbacks.TerminateOnNaN(),
#         tbCallBack
    ]
)

In [ ]:
# h_30_m
keys = ['loss', 'val_loss', 
        'output_1_loss', 'val_output_1_loss', 
        'output_2_loss', 'val_output_2_loss', 
        'output_1_accuracy',  'val_output_1_accuracy', 
        'output_2_mean_absolute_error', 'val_output_2_mean_absolute_error']
fig, axs = plt.subplots(5, 2, figsize=(8, 12))
fig.suptitle('2021.12.6.RMSprop(0.001),loss_weight:0.95, 0.05, epochs=200,batch_size=64')
for idx in range(10):
    i, j = idx // 2, idx % 2
    axs[i][j].plot(h_30_m.history[keys[idx]])
    axs[i][j].set_title(keys[idx])

In [ ]:
keys = ['loss', 'val_loss', 
        'output_1_loss', 'val_output_1_loss', 
        'output_2_loss', 'val_output_2_loss', 
        'output_1_accuracy',  'val_output_1_accuracy', 
        'output_2_mean_absolute_error', 'val_output_2_mean_absolute_error']
fig, axs = plt.subplots(5, 2, figsize=(8, 12))
fig.suptitle('2021.11.30.RMSprop(0.001),loss_weight:0.95, 0.05, epochs=200,batch_size=64')
for idx in range(10):
    i, j = idx // 2, idx % 2
    axs[i][j].plot(h_30_m.history[keys[idx]])
    axs[i][j].set_title(keys[idx])

In [ ]:
h_30_m = model.fit(
    train_x, train_y, 
    epochs=50, batch_size=64, 
    validation_split=0.05,
#     validation_data = (test_x, test_y),
    callbacks = [
#         keras.callbacks.EarlyStopping(
#             monitor='loss', min_delta=0, patience=20, verbose=0, mode='min'),
        tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath ,monitor="val_accuracy", verbose=0,save_best_only=False),
        tf.keras.callbacks.TerminateOnNaN(),
#         tbCallBack
    ]
)

## 2021.11.29

In [ ]:
h_29_n2
keys = ['loss', 'val_loss', 
        'output_1_loss', 'val_output_1_loss', 
        'output_2_loss', 'val_output_2_loss', 
        'output_1_accuracy',  'val_output_1_accuracy', 
        'output_2_mean_absolute_error', 'val_output_2_mean_absolute_error']
fig, axs = plt.subplots(5, 2, figsize=(8, 12))
fig.suptitle('2021.11.29.RMSprop(0.01),loss_weight:0.90, 0.10, epochs=200,batch_size=64')
for idx in range(10):
    i, j = idx // 2, idx % 2
    axs[i][j].plot(h_29_n2.history[keys[idx]])
    axs[i][j].set_title(keys[idx])

In [ ]:
h_29_n2 = model.fit(
    train_x, train_y, 
    epochs=50, batch_size=64, 
    validation_split=0.05,
#     validation_data = (test_x, test_y),
    callbacks = [
#         keras.callbacks.EarlyStopping(
#             monitor='loss', min_delta=0, patience=20, verbose=0, mode='min'),
        tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath ,monitor="val_accuracy", verbose=0,save_best_only=False),
        tf.keras.callbacks.TerminateOnNaN(),
#         tbCallBack
    ]
)

In [ ]:
h_29_n2 = model.fit(
    train_x, train_y, 
    epochs=50, batch_size=64, 
    validation_split=0.05,
#     validation_data = (test_x, test_y),
    callbacks = [
#         keras.callbacks.EarlyStopping(
#             monitor='loss', min_delta=0, patience=20, verbose=0, mode='min'),
        tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath ,monitor="val_accuracy", verbose=0,save_best_only=False),
        tf.keras.callbacks.TerminateOnNaN(),
#         tbCallBack
    ]
)

In [ ]:
keys = ['loss', 'val_loss', 
        'output_1_loss', 'val_output_1_loss', 
        'output_2_loss', 'val_output_2_loss', 
        'output_1_accuracy',  'val_output_1_accuracy', 
        'output_2_mean_absolute_error', 'val_output_2_mean_absolute_error']
fig, axs = plt.subplots(5, 2, figsize=(8, 12))
fig.suptitle('2021.11.29.RMSprop(0.01),loss_weight:0.90, 0.10, epochs=200(50)(50),batch_size=64')
for idx in range(10):
    i, j = idx // 2, idx % 2
    axs[i][j].plot(h_29_n2.history[keys[idx]])
    axs[i][j].set_title(keys[idx])

In [ ]:
keys = ['loss', 'val_loss', 
        'output_1_loss', 'val_output_1_loss', 
        'output_2_loss', 'val_output_2_loss', 
        'output_1_accuracy',  'val_output_1_accuracy', 
        'output_2_mean_absolute_error', 'val_output_2_mean_absolute_error']
fig, axs = plt.subplots(5, 2, figsize=(8, 12))
fig.suptitle('2021.11.29.RMSprop(0.03),loss_weight:0.95, 0.05, epochs=200(50),batch_size=64')
for idx in range(10):
    i, j = idx // 2, idx % 2
    axs[i][j].plot(h_29_n2.history[keys[idx]])
    axs[i][j].set_title(keys[idx])

In [ ]:
keys = ['loss', 'val_loss', 
        'output_1_loss', 'val_output_1_loss', 
        'output_2_loss', 'val_output_2_loss', 
        'output_1_accuracy',  'val_output_1_accuracy', 
        'output_2_mean_absolute_error', 'val_output_2_mean_absolute_error']
fig, axs = plt.subplots(5, 2, figsize=(8, 12))
fig.suptitle('2021.11.29.RMSprop(0.01),loss_weight:0.90, 0.10, epochs=200,batch_size=64')
for idx in range(10):
    i, j = idx // 2, idx % 2
    axs[i][j].plot(h_29_n1.history[keys[idx]])
    axs[i][j].set_title(keys[idx])

In [ ]:
from tensorflow.keras.layers import Dense, LSTM, Masking
from tensorflow.keras.layers import Layer, concatenate
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.losses import CategoricalCrossentropy

from tensorflow.keras.metrics import MeanAbsoluteError, Accuracy

inputs = tf.keras.Input(shape=(train_x.shape[1], train_x.shape[2]))
x = Masking(mask_value=50)(inputs)
x = LSTM(50)(x)

out1 = Dense(50, activation='relu')(x)
out1 = Dense(21, activation='softmax', name='output_1')(out1)

out2 = Dense(50, activation='relu')(x)
out2 = Dense(1, activation='relu', name='output_2')(out2)

model = tf.keras.Model(inputs=[inputs], outputs=[out1, out2])

model.compile(
    optimizer=tf.keras.optimizers.RMSprop(0.05),
    loss={
        'output_1': CategoricalCrossentropy(),
        'output_2': MeanSquaredError()
    },
    loss_weights={
        'output_1': 0.95,
        'output_2': 0.05
    },
    metrics={
        'output_1': Accuracy(),
        'output_2': MeanAbsoluteError()
    }
)

# tbCallBack = tf.keras.callbacks.TensorBoard(
#     log_dir='./woca_logs',  # log 目录                         
#     histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算            
#     batch_size=32,     # 用多大量的数据计算直方图                         
#     write_graph=True,  # 是否存储网络结构图                         
#     write_grads=True,  # 是否可视化梯度直方图                         
#     write_images=True,  # 是否可视化参数                         
#     embeddings_freq=0,                         
#     embeddings_layer_names=None,                         
#     embeddings_metadata=None
# )

checkpoint_filepath = 'MTL-{epoch:02d}-{val_loss:.2f}.hdf5'
h_27_n1 = model.fit(
    train_x, train_y, 
    epochs=200, batch_size=64, 
    validation_split=0.05,
#     validation_data = (test_x, test_y),
    callbacks = [
#         keras.callbacks.EarlyStopping(
#             monitor='loss', min_delta=0, patience=20, verbose=0, mode='min'),
        tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath ,monitor="val_accuracy", verbose=0,save_best_only=False),
        tf.keras.callbacks.TerminateOnNaN(),
#         tbCallBack
    ]
)

In [ ]:

keys = ['loss', 'val_loss', 
        'output_1_loss', 'val_output_1_loss', 
        'output_2_loss', 'val_output_2_loss', 
        'output_1_accuracy',  'val_output_1_accuracy', 
        'output_2_mean_absolute_error', 'val_output_2_mean_absolute_error']
fig, axs = plt.subplots(5, 2, figsize=(8, 12))
fig.suptitle('2021.11.29.RMSprop(0.05),loss_weight:0.95, 0.05, epochs=200,batch_size=64')
for idx in range(10):
    i, j = idx // 2, idx % 2
    axs[i][j].plot(h_27_n1.history[keys[idx]])
    axs[i][j].set_title(keys[idx])

In [ ]:
checkpoint_filepath = 'MTL-{epoch:02d}-{val_loss:.2f}.hdf5'
h_27_n1 = model.fit(
    train_x, train_y, 
    epochs=100, batch_size=64, 
    validation_split=0.05,
#     validation_data = (test_x, test_y),
    callbacks = [
#         keras.callbacks.EarlyStopping(
#             monitor='loss', min_delta=0, patience=20, verbose=0, mode='min'),
        tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath ,monitor="val_accuracy", verbose=0,save_best_only=False),
        tf.keras.callbacks.TerminateOnNaN(),
#         tbCallBack
    ]
)

In [ ]:
checkpoint_filepath = 'MTL-{epoch:02d}-{val_loss:.2f}.hdf5'
h_26_n2 = model.fit(
    train_x, train_y, 
    epochs=100, batch_size=64, 
    validation_split=0.05,
#     validation_data = (test_x, test_y),
    callbacks = [
#         keras.callbacks.EarlyStopping(
#             monitor='loss', min_delta=0, patience=20, verbose=0, mode='min'),
        tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath ,monitor="val_accuracy", verbose=0,save_best_only=False),
        tf.keras.callbacks.TerminateOnNaN(),
        tbCallBack
    ]
)

In [ ]:
keys = ['loss', 'val_loss', 
        'output_1_loss', 'val_output_1_loss', 
        'output_2_loss', 'val_output_2_loss', 
        'output_1_accuracy',  'val_output_1_accuracy', 
        'output_2_mean_absolute_error', 'val_output_2_mean_absolute_error']
fig, axs = plt.subplots(5, 2, figsize=(8, 12))
fig.suptitle('2021.11.26.noon2.RMSprop(0.05),epochs=100,batch_size=64')
for idx in range(10):
    i, j = idx // 2, idx % 2
    axs[i][j].plot(h_26_n2.history[keys[idx]])
    axs[i][j].set_title(keys[idx])

# --------------------分割线--------------------

In [ ]:
from tensorflow.keras.layers import Dense, LSTM, Masking
from tensorflow.keras.layers import Layer, concatenate
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.losses import CategoricalCrossentropy

from tensorflow.keras.metrics import MeanAbsoluteError, Accuracy

inputs = tf.keras.Input(shape=(train_x.shape[1], train_x.shape[2]))
x = Masking(mask_value=50)(inputs)
x = LSTM(50)(x)

out1 = Dense(50, activation='relu')(x)
out1 = Dense(21, activation='softmax', name='output_1')(out1)

out2 = Dense(50, activation='relu')(x)
out2 = Dense(1, activation='relu', name='output_2')(out2)

model = tf.keras.Model(inputs=[inputs], outputs=[out1, out2])

model.compile(
    optimizer=tf.keras.optimizers.RMSprop(0.001),
    loss={
        'output_1': CategoricalCrossentropy(),
        'output_2': MeanSquaredError()
    },
    loss_weights={
        'output_1': 0.5,
        'output_2': 0.5
    },
    metrics={
        'output_1': Accuracy(),
        'output_2': MeanAbsoluteError()
    }
)


checkpoint_filepath = 'MTL-1126n3-{epoch:02d}-{val_loss:.2f}.hdf5'
h_26_n2 = model.fit(
    train_x, train_y, 
    epochs=300, batch_size=64, 
    validation_split=0.05,
#     validation_data = (test_x, test_y),
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='loss', min_delta=0, patience=50, verbose=0, mode='min'),
        tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath ,monitor="val_accuracy", verbose=0,save_best_only=False),
        tf.keras.callbacks.TerminateOnNaN(),
#         tbCallBack
    ]
)